In [6]:
%pip install C:/projects/tinygrad

Processing c:\projects\tinygrad
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for tinygrad: filename=tinygrad-0.10.1-py3-none-any.whl size=1438385 sha256=9fa902f476bfd7884cab21276f3a77644b65ec3eb30275a3384258fdae17cbc6
  Stored in directory: C:\Users\Nathan\AppData\Local\Temp\pip-ephem-wheel-cache-16g_h9ua\wheels\31\6a\4d\db7bda76389de9857b72005b69e072a50366eb4ebc01fa8dc1
Successfully built tinygrad
Note: you may need to restart the kernel to use updated packages.


In [9]:
import tinygrad

from tinygrad import Device

print(Device.DEFAULT)

GPU


In [6]:
from tinygrad import Tensor, nn, Context

In [5]:
Tensor([1, 2, 3]).data().tolist()

[1, 2, 3]

In [7]:
with Context(DEBUG=4):
    x = Tensor([1, 1])
    y = Tensor([2, 2])
    z = x + y
    print(z.tolist())

*** GPU        5 copy        8,     GPU <- PYTHON          arg  2 mem  0.00 GB tm    717.10us/     0.72ms (     0.00 GFLOPS    0.0|0.0     GB/s) 
*** GPU        6 copy        8,     GPU <- PYTHON          arg  2 mem  0.00 GB tm    407.50us/     1.12ms (     0.00 GFLOPS    0.0|0.0     GB/s) 
E_2
 0: (2,)                      int.ptr(2)           (1,)
 1: (2,)                      int.ptr(2)           (1,)
 2: (2,)                      int.ptr(2)           (1,)
[Opt(op=OptOps.LOCAL, axis=0, arg=2)]
__kernel void E_2(__global int* data0, __global int* data1, __global int* data2) {
  int lidx0 = get_local_id(0); /* 2 */
  int val0 = *(data1+lidx0);
  int val1 = *(data2+lidx0);
  *(data0+lidx0) = (val0+val1);
}
*** GPU        7 E_2                                       arg  3 mem  0.00 GB tm      1.08us/     1.13ms (     0.00 GFLOPS    0.0|0.0     GB/s) ['__add__', 'tolist']
*** CLANG      8 copy        8,   CLANG <- GPU             arg  2 mem  0.00 GB tm    274.00us/     1.40ms (     0.00 

In [9]:
with Context(DEBUG=4):
    x = Tensor.ones(1000)
    y = Tensor.ones(1000)
    z = x + y
    print(z.tolist())

E_125_2_4
 0: (125, 2, 4)               float.ptr(1000)      (8, 4, 1)
[Opt(op=OptOps.UPCAST, axis=0, arg=4), Opt(op=OptOps.LOCAL, axis=0, arg=2)]
__kernel void E_125_2_4(__global float* data0) {
  int gidx0 = get_group_id(0); /* 125 */
  int lidx0 = get_local_id(0); /* 2 */
  *((__global float4*)((data0+((gidx0<<3)+(lidx0<<2))))) = (float4)(2.0f,2.0f,2.0f,2.0f);
}
*** GPU        9 E_125_2_4                                 arg  1 mem  0.00 GB tm      3.20us/     1.40ms (     0.00 GFLOPS    1.2|1.2     GB/s) ['tolist']
*** CLANG     10 copy     4000,   CLANG <- GPU             arg  2 mem  0.00 GB tm    361.90us/     1.76ms (     0.00 GFLOPS    0.0|0.0     GB/s) ['tolist']
[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0,

In [10]:
class Model:
    def __init__(self):
        self.l1 = nn.Conv2d(1, 32, kernel_size=(3, 3))
        self.l2 = nn.Conv2d(32, 64, kernel_size=(3, 3))
        self.l3 = nn.Linear(1600, 10)
    
    def __call__(self, x:Tensor) -> Tensor:
        x = self.l1(x).relu().max_pool2d((2, 2))
        x = self.l2(x).relu().max_pool2d((2, 2))
        return self.l3(x.flatten(1).dropout(0.5))

In [11]:
Model()

In [12]:
from tinygrad.nn.datasets import mnist
x_train, y_train, x_test, y_test = mnist()

x_train.shape

https://storage.googleapis.com/cvdf-datasets/mnist/train-images-idx3-ubyte.gz: 4
https://storage.googleapis.com/cvdf-datasets/mnist/train-labels-idx1-ubyte.gz: 6
https://storage.googleapis.com/cvdf-datasets/mnist/t10k-images-idx3-ubyte.gz: 7.
https://storage.googleapis.com/cvdf-datasets/mnist/t10k-labels-idx1-ubyte.gz: 10


(60000, 1, 28, 28)

In [14]:
model = Model()

acc = (model(x_test).argmax(axis = 1) == y_test).mean()

print(acc.item())

0.0778999999165535


In [16]:
optim = nn.optim.Adam(nn.state.get_parameters(model))

batch_size = 128

def step():
    Tensor.training = True
    samples = Tensor.randint(batch_size, high=x_train.shape[0])

    x, y = x_train[samples], y_train[samples]

    optim.zero_grad()

    loss = model(x).sparse_categorical_crossentropy(y).backward()
    optim.step()

    return loss

In [17]:
import timeit

timeit.repeat(step, repeat=5, number=1)

[17.196870899992064,
 2.979917899996508,
 0.1161939000012353,
 0.11727349998545833,
 0.11661029999959283]

In [18]:
acc = (model(x_test).argmax(axis = 1) == y_test).mean()

print(acc.item())

0.3449999988079071


In [19]:
from tinygrad import TinyJit

jit_step = TinyJit(step)

In [21]:
timeit.repeat(jit_step, repeat=100, number=1)

[0.0021726999839302152,
 0.001881500007584691,
 0.0021829000033903867,
 0.0017128000035881996,
 0.00157789999502711,
 0.0013420000032056123,
 0.001292400003876537,
 0.0014533000066876411,
 0.0028750000055879354,
 0.0014076000079512596,
 0.001264000020455569,
 0.0012578000023495406,
 0.0013394999841693789,
 0.001320600014878437,
 0.001364399999147281,
 0.0016627000004518777,
 0.001295199996093288,
 0.0012296000204514712,
 0.001260000019101426,
 0.0012584000069182366,
 0.0015312999894376844,
 0.0013288999907672405,
 0.001277000003028661,
 0.0012571999977808446,
 0.0014174999960232526,
 0.0012271000014152378,
 0.0012771000037901103,
 0.001264699996681884,
 0.0013056000170763582,
 0.0012614000006578863,
 0.001299499999731779,
 0.0013464999792631716,
 0.001246399973751977,
 0.0011756999883800745,
 0.001183400017907843,
 0.0017502999980933964,
 0.0029174000083003193,
 0.0015038999845273793,
 0.0011666000063996762,
 0.0011547999747563154,
 0.0012123000051360577,
 0.0014792000001762062,
 0.001

In [22]:
acc = (model(x_test).argmax(axis = 1) == y_test).mean()

print(acc.item())

0.8924999833106995
